In [ ]:
from random import seed, random, randrange
from math import exp
from csv import reader
import numpy as np
import time
import sys

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler # Preprocessing
from sklearn.metrics import mean_squared_error as MSE
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import accuracy_score, recall_score

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#df_data = pd.read_csv('all_data2.csv')
df_data = np.loadtxt('all_data2.csv', delimiter=',', skiprows=1)
print(df_data.shape)

In [ ]:
var = 'map_idx'
data = pd.concat([df_data['pm2_5'], df_data[var]], axis=1)
data.plot.scatter(x=var, y='pm2_5', ylim=(0,4));

In [ ]:
#X, y = df_data.drop(["pm2_5"], axis = 1), (df_data["pm2_5"])
X = df_data[:, 1:]
y = df_data[:,0]
print("Training set shape (%d,%d)" %X.shape)
#y = y.values.reshape(-1, 1)
y = y.reshape(-1, 1)

In [ ]:
# convert y to binary class
y_ = (y >= 1.5).astype(int)
print(y_)

In [ ]:
# convert y to multiple classes
print(y)
y_ = np.copy(y)
y_[np.argwhere(y < 0.5)[:,0]] = 0
y_[np.argwhere(np.logical_and(y >= 0.5, y < 1.5))[:,0]] = 1
y_[np.argwhere(np.logical_and(y >= 1.5, y < 2.5))[:,0]] = 2
y_[np.argwhere(y >= 2.5)[:,0]] = 3
print(y_)

In [ ]:
#Label categorical values
cat_features = [0, 0]
for feature in cat_features:
    le = LabelEncoder()
    X[:, feature]  = le.fit_transform(X[:, feature])
    print(X[:, feature])
    # Perform one hot encoding
    ohe = OneHotEncoder(sparse = False)
    # Name columns
    columns = [str(feature) + '_' + str(class_) for class_ in le.classes_]
    #Drop first column to avoid dummy variable trap
    X_dummies = ohe.fit_transform(X[:, feature].reshape(-1,1))[:,1:]
    print(X_dummies.shape)
    # Drop original feature
    #X = X.drop(feature, axis=1)
    X = np.delete(X, feature, axis=1)
    #X = pd.concat([X, X_dummies], axis=1)
    X = np.concatenate((X, X_dummies), axis=1)
    
print("\n After One Hot Encoding")
print("Training set shape (%d,%d)" %X.shape)

In [ ]:
dataset = X
target = y_
train_set, validate_set, y_train, y_validate = train_test_split(dataset, target, test_size=0.20, random_state=100)

In [ ]:
#np.savetxt('train_set.data', train_set, fmt='%.2f')
#np.savetxt('validate_set.data', validate_set, fmt='%.2f')
train_set.shape

In [ ]:
X_train = train_set
X_validate = validate_set

clf = MLPClassifier(hidden_layer_sizes=(1000, ), max_iter=10, alpha=1e-2, random_state=1, verbose=True)
clf.fit(X_train, y_train)

In [ ]:
y_predict = clf.predict(X_validate)
print("accuracy = %f" %accuracy_score(y_validate, y_predict))
print("recall = %f" %recall_score(y_validate, y_predict))

In [ ]:
print np.count_nonzero(y_validate == 1)
print np.count_nonzero(y_predict == 1)